In [1]:
import torch
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
from tqdm import tqdm
import sys
import colorama
from colorama import Fore
from transformers import BertForQuestionAnswering, AdamW
import numpy as np

In [2]:
train_path = "./train-v2.0.json"
dev_path = "./dev-v2.0.json"

import os

if not (os.path.isfile(train_path)):
  !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json


if not (os.path.isfile(dev_path)):
  !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2022-03-24 15:11:52--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   250MB/s    in 0.2s    

2022-03-24 15:11:54 (250 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2022-03-24 15:11:54--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [3]:
def json_to_df(file_name):
    # Open .json file
    with open(file_name, 'r') as f:
        squad_dict = json.load(f)

    ids=[]
    context = []
    title=[]
    question = []
    answer = []
    answer_start=[]

    count=0
    # Search for each passage, its question and its answer
    for group in squad_dict['data']:
        t=group['title']
        # print("GROUP:")
        # print(group)
        for passage in group['paragraphs']:
            c = passage['context']  #keep the context
            # print("PASSAGE:")
            # print(passage)
            for qa in passage['qas']:
                q = qa['question'] #keep the question
                id_=qa["id"]        #keep the id 
                # check if we need to extract from 'answers' or 'plausible_answers'
                if 'plausible_answers' in qa.keys():
                    answer_type = 'plausible_answers'
                else:
                    answer_type = 'answers'
                #for every answer    
                for an in qa[answer_type]:
                    # Store every id, title, context, question and its answer to the lists
                    ids.append(id_)
                    title.append(t)
                    context.append(c)
                    question.append(q)
                    answer.append(an['text'])
                    answer_start.append(an['answer_start'])
                    count+=1
                    
    #the number of rows that will be produced
    print(count)

    tr_df = pd.DataFrame(columns=['Id','title','context','question','answer', 'ans_start']) # Creating empty DataFrame.   
    tr_df.Id = ids
    tr_df.title = title           #intializing list values to the DataFrame.
    tr_df.context = context
    tr_df.question = question
    tr_df.answer = answer
    tr_df.ans_start = answer_start
    
    #because in dev set we come accross many duplicate answers. I drop the duplicates and keep the first one
    #final_df = tr_df.drop_duplicates(keep='first',ignore_index=True)  # Dropping duplicate rows from the create Dataframe.
    return(tr_df)


# train_df=json_to_df("./train-v2.0.json")
# val_df=json_to_df("./dev-v2.0.json")

In [4]:
#a function that ajust the position of the start and end of the answer
#Probably this function won't be used
def true_pos(row):
    text=row['context']
    true_answer=row['answer']
    start=row['ans_start']
    end=row['ans_end']
    
    if text[start:end] != true_answer:
        print(row['Id'])
    # When the real answer is more by one character
    if text[start-1:end-1] == true_answer:
        row['ans_start']= row['ans_start']-1
        row['ans_end']= row['ans_end']-1
        #print(row['Id'])
    # When the real answer is more by two characters  
    elif text[start-2:end-2] == true_answer:
        row['ans_start']= row['ans_start']-2
        row['ans_end']= row['ans_end']-2
        #print(row['Id'])
    # if text[start:end] == true_answer:
    #     row['Real_answer']=True
    return row

In [5]:
from transformers import BertTokenizerFast,AdamW,BertForQuestionAnswering
tk = BertTokenizerFast.from_pretrained("bert-base-uncased")

M_L=512 #max length
B_S=8   #batch size
EPOCHS_NUM=2        #the number of epochs

#Tokenize the text pair of contexts-questions. 
def my_tokenizer(my_df):
    my_contexts=my_df.context.values.tolist()
    my_questions=my_df.question.values.tolist()

    my_encodings = tk(my_contexts, my_questions,  truncation=True, padding=True, return_attention_mask = True, max_length=M_L, return_tensors = 'pt')
    #return the produced encodings
    return my_encodings


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
#A function to convert the start and end characters position to TOKEN position
def convert_to_token_positions(encodings, my_df):
  starts=my_df.ans_start.values.tolist()
  ends=my_df.ans_end.values.tolist()
  start_positions = []
  end_positions = []

  start_pos=0
  end_pos=0
  #for every start position
  for i in range(len(starts)):
    #convert the character position to token position
    start_pos=encodings.char_to_token(i, starts[i])
    end_pos=encodings.char_to_token(i, ends[i])

    # if start position is None, the answer passage has been truncated
    if start_pos is None:
      #make start position equal to the last token
      start_pos = tk.model_max_length
      
    shift = 1
    while end_pos is None:
      end_pos = encodings.char_to_token(i, ends[i] - shift)
      shift += 1

    start_positions.append(start_pos)
    end_positions.append(end_pos)
  return start_positions, end_positions

In [7]:
#a function to create the data loaders both train and val
def my_Dataloader(input_ids, attention_mask, token_type_ids, starts, ends):

    t_starts=torch.Tensor(starts)
    t_ends=torch.Tensor(ends)
    my_dataset=torch.utils.data.TensorDataset(input_ids, attention_mask, t_starts, t_ends,token_type_ids)
    my_dataloader = torch.utils.data.DataLoader(my_dataset, batch_size=B_S,shuffle=True) #shuffle=True

    return my_dataloader

In [8]:
import re
import string

#a function to clear the text. This will be used in the metrics calculation to see how close the answers are
def normalize_text(text):
    #convert to lower
    text=text.lower()
    #remove punctuation
    pncts=set(string.punctuation)
    text=''.join(ch for ch in text if ch not in pncts)
    #remove articles
    regex= re.compile(r'\b(a|an|the)\b', re.UNICODE)
    text=re.sub(regex, ' ', text)
    #white space fix
    text= ' '.join(text.split())
    return text
    
#A function that takes the input_ids, the start and end pointer, and returns the answer in text format
def find_answer(input_ids, start, end):
    # print(input_ids)
    # print(start)
    # print(end)
    
    answer = tk.convert_tokens_to_string(tk.convert_ids_to_tokens(input_ids[start.int():end.int()])) #+1
    #print(answer)
    return normalize_text(answer)

#Function to compute F1 score
def compute_f1(prediction, target):
    pred_tokens = prediction.split()
    truth_tokens = target.split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)


#function to compute f1 score of the whole batch
def compute_batch_f1(input_ids, predicted_start, predicted_end, target_start, target_end ):
    total_f1=0
    b_s=len(input_ids)
    for i in range(b_s):
        predicted_answer=find_answer(input_ids[i], predicted_start[i], predicted_end[i])
        target_answer=find_answer(input_ids[i], target_start[i], target_end[i])
        total_f1+=compute_f1(predicted_answer, target_answer)

    return total_f1/b_s

#function to calculate the accuracy given the predictions of each class and the truly label
def answer_accuracy(input_ids, start_predictions, end_predictions, start_targets, end_targets):
    #convert the results to probabilities
    # start_pred_softmax = torch.log_softmax(start_predictions, dim = 1)
    # end_pred_softmax = torch.log_softmax(end_predictions, dim = 1)
    #take the start and end position with the highest probability
    start_predicts=torch.argmax(start_predictions,1)
    end_predicts=torch.argmax(end_predictions,1)
    corrects_found=0
    #parse every start and end position and see if the text is the exact same with the predicted one
    for i in range(len(input_ids)):
        #calculate the predicted answer
        predicted_answer=find_answer(input_ids[i], start_predicts[i], end_predicts[i])
        #calculate the target answer
        target_answer=find_answer(input_ids[i], start_targets[i], end_targets[i])
        # print("Predicted answer: ",predicted_answer)
        # print("Targeted answer: ",target_answer)
        #if the answers are the same then increase the correxts ones
        if predicted_answer==target_answer:
            corrects_found+=1
            # print("IT IS CORRECT!!")
    #corrects_found=(((torch.squeeze(start_targets) == start_predicts) and (torch.squeeze(end_targets) == end_predicts))).float()
    return corrects_found/len(input_ids)

In [9]:


#function to train the model
def training_f(train_df, train_dataloader, model):
    epochs=[]
    #TRAIN OF THE MODEL
    for epoch in range(EPOCHS_NUM):
        #initialising the training bar
        training_pbar = tqdm(total=len(train_df),
                            position=0, leave=True,
                            file=sys.stdout, bar_format="{l_bar}%s{bar}%s{r_bar}" % (Fore.GREEN, Fore.RESET))
        batch_losses = []   #a list with the mean losses for each batch
        batch_accuracies = []   #a list with the mean accuracies for each batch
        epochs.append(epoch)
        model.train()
        for batch in train_dataloader:
            #retrieve all the useful info for each batch
            train_input_ids = batch[0].to(device)   
            train_input_mask = batch[1].to(device)
            train_starts = batch[2].to(device)
            train_end= batch[3].to(device)
            train_token_type_ids=batch[4].to(device)
            #Delete previously stored gradients
            optimizer.zero_grad()

            #forward of the model
            loss, output_start, output_end = model(train_input_ids, 
                                #token_type_ids=train_token_type_ids,
                                start_positions=train_starts.long(),
                                end_positions=train_end.long(),
                                attention_mask=train_input_mask,
                                return_dict=False)
            #collecth the loss of the batch
            batch_losses.append(loss.item())
            
            #Perform backpropagation starting from the loss calculated in this epoch
            loss.backward()
            #gradient clipping do not exceed 20
            torch.nn.utils.clip_grad_value_(model.parameters(), 20)
            #Update model's weights based on the gradients calculated during backprop
            optimizer.step()
            
            #calculate the accuracy
            batch_acc=answer_accuracy(train_input_ids, output_start, output_end, train_starts, train_end)
            batch_accuracies.append(batch_acc)
            #update the training training bar
            training_pbar.update(train_input_ids.size(0))
        #close the training bar
        training_pbar.close()
        #calculate the loss and the accuracy for each epoch
        loss=sum(batch_losses)/len(train_dataloader)
        print(f"Epoch {epoch:3}: Loss = {loss:.5f}")
        acc=sum(batch_accuracies)/len(train_dataloader)
        print(f"Accuracy = {acc:.5f}")

In [10]:
def evaluation_f(val_df, val_dataloader, model):
    #VALIDATION OF MODEL
    with torch.no_grad():
        #initialize the validation bar
        validation_pbar = tqdm(total=len(val_df),
                                position=0, leave=True,
                                file=sys.stdout, bar_format="{l_bar}%s{bar}%s{r_bar}" % (Fore.BLUE, Fore.RESET))
        
        val_batch_losses = []       #a list with the mean losses for each batch
        val_batch_accuracies = []       #a list with the mean accuracies for each batch
        f_ones_scores=[]                #a list with the mean f1 for each batch
        model.eval()
        #takes each validation batch
        for val_batch in val_dataloader:
            #retrieve all the useful info for each batch
            val_input_ids = val_batch[0].to(device)
            val_input_mask = val_batch[1].to(device)
            val_starts = val_batch[2].to(device)
            val_ends= val_batch[3].to(device)
            val_token_type_ids=val_batch[4].to(device)
            #make the predictions based on the current trained model
            val_loss, output_start, output_end = model(val_input_ids, 
                            #token_type_ids=val_token_type_ids,
                            start_positions=val_starts.long(),
                            end_positions=val_ends.long(),
                            attention_mask=val_input_mask,
                            return_dict=False)
            #calculate the losses
            val_batch_losses.append(val_loss)
            #convert the results to probabilities
            # start_pred_softmax = torch.log_softmax(output_start, dim = 1)
            # end_pred_softmax = torch.log_softmax(output_end, dim = 1)
            #take the start and end position with the highest probability
            start_predict=torch.argmax(output_start,1)
            end_predict=torch.argmax(output_end,1)

            #calculate the accuracy
            val_acc = answer_accuracy(val_input_ids, output_start, output_end, val_starts, val_ends)
            val_batch_accuracies.append(val_acc)

            #calculate f1 score    
            f1_score=compute_batch_f1(val_input_ids, start_predict, end_predict, val_starts, val_ends )
            f_ones_scores.append(f1_score)

            #update the validation bar to check progress
            validation_pbar.update(val_input_ids.size(0))
            
        #close the validation bar
        validation_pbar.close()
        print("F1 score: ",sum(f_ones_scores)/len(f_ones_scores))
        print("Accuracy score: ",sum(val_batch_accuracies)/len(val_batch_accuracies))
        print("Loss: ", sum(val_batch_losses)/len(val_batch_losses))

In [11]:
t_p="./train-v2.0.json"
v_p="./dev-v2.0.json"

#a function that will train the model based on a set
def train(path, model):
    train_df=json_to_df(path)
    train_df["ans_end"]=train_df['ans_start']+train_df['answer'].str.len() #adding the end position
    train_df.apply(lambda x: true_pos(x), axis=1)

    train_df=train_df[:1000]

    #take the train encodings
    train_encds = my_tokenizer(train_df)
    #retrieve input_ids attention_mask and token_type_ids
    i_ids, a_mask, t_type_ids=train_encds['input_ids'], train_encds['attention_mask'], train_encds['token_type_ids']
    
    train_ans_start, train_ans_end = convert_to_token_positions(train_encds, train_df)

    train_dl=my_Dataloader(i_ids, a_mask, t_type_ids, train_ans_start, train_ans_end)

    training_f(train_df,train_dl,model)

In [12]:
#a function to evaluate a set based on a model
def eval(path, model):
    eval_df=json_to_df(path)
    eval_df["ans_end"]=eval_df['ans_start']+eval_df['answer'].str.len() #adding the end position
    eval_df.apply(lambda x: true_pos(x), axis=1)

    eval_df=eval_df[:100]

    #take the train encodings
    eval_encds = my_tokenizer(eval_df)
    #retrieve input_ids attention_mask and token_type_ids
    i_ids, a_mask, t_type_ids=eval_encds['input_ids'], eval_encds['attention_mask'], eval_encds['token_type_ids']
    
    val_ans_start, val_ans_end = convert_to_token_positions(eval_encds, eval_df)

    eval_dl=my_Dataloader(i_ids, a_mask, t_type_ids, val_ans_start, val_ans_end)

    evaluation_f(eval_df,eval_dl,model)

In [13]:
from transformers import BertForQuestionAnswering
import numpy as np

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Load BertForSequenceClassification, the pretrained BERT model
model = BertForQuestionAnswering.from_pretrained(
    "bert-base-uncased" # Use the 12-layer BERT model, with an uncased vocab.
).to(device)

#Define Hyperparameters
learning_rate = 3e-5 #0.00003
#using Adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 1e-3)  #Adam optimizer

Using cuda device


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [14]:
# pth2='./squad_bert_fine_tuned_model.pt'
# model.load_state_dict(torch.load(pth2))

In [15]:
train(t_p, model)

130319
100%|██████████| 1000/1000 [00:54<00:00, 18.19it/s]
Epoch   0: Loss = 4.16275
Accuracy = 0.06900
100%|██████████| 1000/1000 [00:53<00:00, 18.59it/s]
Epoch   1: Loss = 2.85147
Accuracy = 0.20800


In [16]:
#SAVE MODEL WEIGHTS
pth='./model_weights.pt'
torch.save(model.state_dict(), pth)

In [17]:
eval(v_p, model)

26232
100%|██████████| 100/100 [00:01<00:00, 94.47it/s]
F1 score:  0.19487318979793897
Accuracy score:  0.14423076923076922
Loss:  tensor(3.2358, device='cuda:0')


In [18]:
context="Anafi is a small island in the Cyclades, located to the east of Santorini. According to mythology, Anafi emerged from the Aegean Sea when the Argonauts prayed to Apollo, \
the god of music and light, to save them from the angry sea. That is why the Argonauts built a temple dedicated to this god, whose worship was widespread on the island.\
Remains of this temple can still be found near the Monastery of Panagia Kalamiotissa. It is said that Anafi was first occupied by the Phoenicians and later by the Dorians. \
Following them, in the 5th century BC, it was conquered by Athens. During the Middle Ages, it became part of the Duchy of Naxos and, later on, the stronghold of several\
Venetian royal families. In 1207 AD, Anafi came under Frankish rules The Castle of Krispi (a.k.a. Glezos Tower) built on the top of the hill of Chora was characteristic of this period, \
as it used to be the shelter of the inhabitants against pirates and enemies. In the 14th century, the Ottomans took control over the island and established a severe regime. \
In 1537, the history of Anafi is marked by the attack of pirate Barbarosa, who sold all the male inhabitants as slaves and slaughtered the women and children.\
The island was liberated by the Turkish occupation in 1830, along with the other Cyclades islands. Since then, its economy is poor due to its small size and few inhabitants. \
That is why most of the inhabitants in the 1830s-1940s moved to the capital and settled in the district of Plaka, right in the center of Athens. Their new neighborhood was \
named Anafiotika and today it is the loveliest and most picturesque area of Plaka. Today, Anafi bases its economy mostly on fishing, agriculture, and tourism. In fact, only \
    over the last years has tourism developed in Anafi and the tourists that go there are looking for extreme peace and quiet, as the tourist amenities on the island are few."

question="When Anafi was liberated by the Turkish?"


def predict_answer(context, question, model):
    m_l=256

    ex_encodings = tk(context,question,  truncation='only_first', padding=True, return_attention_mask = True, max_length=m_l, return_tensors = 'pt')
    ex_input_ids=ex_encodings['input_ids']
    ex_attention_masks=ex_encodings['attention_mask']
    ex_token_type_ids=ex_encodings['token_type_ids']

    with torch.no_grad():
        model.eval()

        val_input_ids = ex_input_ids.to(device)
        val_input_mask = ex_attention_masks.to(device)
        val_token_type_ids=ex_token_type_ids.to(device)
        #make the predictions based on the current trained model
        output_start, output_end = model(val_input_ids, 
                        token_type_ids=val_token_type_ids,
                        attention_mask=val_input_mask,
                        return_dict=False)
        #the label that the tweet belongs to will be the one with the highest propability
        start_predict=torch.argmax(output_start,1)
        end_predict=torch.argmax(output_end,1)

        answer= tk.convert_tokens_to_string(tk.convert_ids_to_tokens(val_input_ids[0][start_predict.int():end_predict.int()]))
    return answer

In [19]:
context="Arya arrives in Braavos and is accepted into the 'House of Black and White' by Jaqen H'ghar, who calls himself and\
his colleagues, 'no one'. Jaime recruits Bronn to travel to Dorne with him to retrieve Jaime's niece/daughter Myrcella,\
who is now betrothed to Trystane Martell, Prince Doran's son. Podrick recognises Littlefinger and Sansa in a tavern;\
Brienne offers Sansa her protection but is rebuffed. Littlefinger insists Brienne come with them, but she refuses and escapes\
with Podrick, then secretly follows Sansa. Stannis offers Jon legitimacy and lordship of Winterfell if he leaves the Night's\
Watch and supports him, but Jon keeps his vow to the Night's Watch; Samwell nominates Jon as Lord Commander against\
Alliser Thorne; Maester Aemon casts his ballot for Jon, breaking a tie vote.Daenerys incites a riot by executin\
a former slave who murdered a Sons of the Harpy prisoner prior to his trial.Drogon reappears, but soon leaves."

question="How does Jaqen H'ghar call himself?"

predict_answer(context,question,model)

'no one'